# Templating input cells with [`jinja2`](http://jinja.pocoo.org/docs/2.10/)

This notebook demonstrates and exports the ability to place real into code cells before executing code. This idea can create parameterized reports.

    %reload_ext deathbeds.__Jinja2__

In [1]:
    from deathbeds.__Markdown__ import CallableTransformer
    from jinja2 import Template

AttributeError: 'NoneType' object has no attribute 'name'

* Use the callable transformer from the previous [Markdown input transformer](2018-07-06-Markdown-code-cells.ipynb)


In [2]:
    class TemplateTransformer(CallableTransformer):
        def __call__(self, str):
            from IPython import get_ipython
            return Template(str).render(**get_ipython().user_ns)

* Export the Ipython extension function.

In [3]:
    def load_ipython_extension(ip):
        ip.input_transformer_manager.physical_line_transforms.insert(0, TemplateTransformer())

* In the proof of concept it should be possible to combine the [Markdown transform](2018-07-06-Markdown-code-cells.ipynb) with the new template transformer.

In [4]:
    Ø = __name__ == '__main__'
    if Ø:
        %reload_ext deathbeds.__Markdown__
        %reload_ext deathbeds.__Jinja2__

In [5]:
# Markdown and templates can now mix

    a = 10

# Markdown and templates can now mix

    a = 10


In [7]:
> This is a template value of `a` which is {{a}}

> This is a template value of `a` which is 10
